# Replicación de objetos

**[LINK](https://docs.aws.amazon.com/es_es/AmazonS3/latest/userguide/replication.html)**

Con la replicación es posible copiar objetos entre buckets de Amazon S3 de forma automática y asincrónica. Los buckets que están configurados para reproducción de objetos pueden pertenecer a la misma Cuenta de AWS o a cuentas diferentes. Puede replicar objetos en un solo bucket de destino o en varios buckets de destino. Los buckets de destino pueden estar en diferentes Regiones de AWS o dentro de la misma región que el bucket de origen.

Para replicar automáticamente objetos nuevos a medida que se escriben en el bucket, utilice la replicación en directo, como la replicación entre regiones (CRR). Para replicar objetos existentes en un bucket diferente bajo demanda, utilice la replicación por lotes de S3. Para obtener más información sobre la replicación de objetos existentes, consulte Cuándo utilizar la replicación por lotes de S3.

Para habilitar la CRR, agregue una configuración de replicación al bucket de origen. La configuración mínima, debe proporcionar lo siguiente:

* El bucket o buckets de destino en los que desea que Amazon S3 replique objetos
* Un rol de AWS Identity and Access Management (IAM) que Amazon S3 pueda asumir para reproducir objetos en su nombre

La replicación puede ayudarle a hacer lo siguiente:

* Replicar objetos reteniendo los metadatos: puede utilizar la replicación para realizar copias de los objetos en las que se retengan todos los metadatos, como las horas de creación del objeto original y los ID de versión. Esta capacidad es importante si debe asegurarse de que la réplica sea idéntica al objeto de origen.

* Replicar objetos en diferentes clases de almacenamiento: puede utilizar la replicación para colocar objetos directamente en S3 Glacier Flexible Retrieval, S3 Glacier Deep Archive u otra clase de almacenamiento en los buckets de destino. También puede replicar los datos en la misma clase de almacenamiento y utilizar las políticas de ciclo de vida en los buckets de destino para mover objetos a una clase de almacenamiento con menos actividad conforme adquieren antigüedad.

* Mantener copias de objetos con distintos propietarios: independientemente de quién sea el propietario del objeto de origen, puede indicar a Amazon S3 que cambie la propiedad de la réplica a la Cuenta de AWS que posee el bucket de destino. Esto se conoce como la opción de invalidación del propietario. Puede usar esta opción para restringir el acceso a las réplicas de objetos.

* Mantener los objetos almacenados en varias Regiones de AWS: para garantizar las diferencias geográficas en el lugar donde se guardan los datos, puede establecer varios buckets de destino en diferentes Regiones de AWS. Esta característica podría ayudarle a cumplir ciertos requisitos de conformidad.

* Replicar objetos en 15 minutos: para replicar sus datos en la misma Región de AWS o en distintas regiones dentro de un periodo predecible, puede utilizar el control de tiempo de replicación de S3 (S3 RTC). S3 RTC replica el 99,99 % de los objetos nuevos almacenados en Amazon S3) en un plazo de 15 minutos (con el respaldo de un acuerdo de nivel de servicio). Para obtener más información, consulte Cumplimiento de los requisitos de conformidad mediante el control de tiempo de replicación de S3 (S3 RTC).
* Sincronizar buckets, replicar objetos existentes y replicar objetos que fallaron o se replicaron anteriormente: para sincronizar buckets y replicar objetos existentes, utilice la replicación por lotes como acción de replicación bajo demanda. Para obtener más información sobre cuándo utilizar la replicación por lotes, consulte Cuándo utilizar la replicación por lotes de S3.

* Replicar objetos y realizar una conmutación por error a un bucket en otra Región de AWS: para mantener todos los metadatos y objetos sincronizados entre los buckets durante la replicación de datos, utilice las reglas de replicación bidireccional (también conocidas como replicación bidireccional) antes de configurar los controles de conmutación por error de los puntos de acceso multirregionales de Amazon S3. Las reglas de replicación bidireccional ayudan a garantizar que, cuando se escriben datos en el bucket de S3 al que se transfiere el tráfico por error, esos datos se repliquen de nuevo en el bucket de origen.

In [17]:
import boto3
import json
import random
from botocore.exceptions import ClientError

region_aws = 'us-east-1'
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

### Crear dos bucket

In [5]:
bucket_orig = 'cookbook-orig-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_orig)
bucket_dest = 'cookbook-dest-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_dest)
print(f"Bucket de origen: {bucket_orig} \nBucket de destino: {bucket_dest}")

### Habilitar el control de versiones en S3

In [15]:
bucket_versioning = s3.BucketVersioning(bucket_orig)
response = bucket_versioning.enable()
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Versioning habilitado en el bucket {}'.format(bucket_orig))
    
bucket_versioning = s3.BucketVersioning(bucket_dest)
response = bucket_versioning.enable()
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Versioning habilitado en el bucket {}'.format(bucket_dest))

Versioning habilitado en el bucket cookbook-885384
Versioning habilitado en el bucket cookbook-112194


### Cree un rol de IAM. 
Especifique este rol en la configuración de replicación que agregue al bucket de origen más adelante. Amazon S3 asume este rol para replicar objetos en su nombre.

In [16]:
s3_trust_policy = {
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Principal":{
            "Service":"s3.amazonaws.com"
         },
         "Action":"sts:AssumeRole"
      }
   ]
}

In [18]:
# Cretae role
iam = boto3.client('iam')
role_name = 's3-replication-role'

try:
    response = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(s3_trust_policy),
        Description='Role for S3 replication'
    )
    print('Role {} created'.format(role_name))
except ClientError as e:
    print("Unexpected error: %s" % e)

Role s3-replication-role created


In [30]:
s3_role_permissions_policy = {
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Action":[
            "s3:GetObjectVersionForReplication",
            "s3:GetObjectVersionAcl",
            "s3:GetObjectVersionTagging"
         ],
         "Resource":[
            f"arn:aws:s3:::{bucket_orig}/*"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ListBucket",
            "s3:GetReplicationConfiguration"
         ],
         "Resource":[
            f"arn:aws:s3:::{bucket_orig}"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ReplicateObject",
            "s3:ReplicateDelete",
            "s3:ReplicateTags"
         ],
         "Resource":f"arn:aws:s3:::{bucket_dest}/*"
      }
   ]
}

In [32]:
# Attach policy to role

try:
    response = iam.put_role_policy(
        RoleName=role_name,
        PolicyName='s3-replication-policy',
        PolicyDocument=json.dumps(s3_role_permissions_policy)
    )
    print('Policy attached to role {}'.format(role_name))
except ClientError as e:
    print(e.response['Error']['Message'])

Policy attached to role s3-replication-role


### Agregue la configuración de replicación al bucket source

In [41]:
# Get role ARN
arn_role = iam.get_role(RoleName=role_name)['Role']['Arn']

replication_configuration = {
  "Role": arn_role,
  "Rules": [
    {
      "Status": "Enabled",
      "Priority": 1,
      "DeleteMarkerReplication": { "Status": "Disabled" },
      "Filter" : { "Prefix": "Tax"},
      "Destination": {
        "Bucket": f"arn:aws:s3:::{bucket_dest}"
      }
    }
  ]
}

In [42]:
# Put bucket replication configuration
try:
    response = s3_client.put_bucket_replication(
        Bucket=bucket_orig,
        ReplicationConfiguration=replication_configuration
    )
    print('Replication configuration created')
except ClientError as e:
    print(e.response['Error']['Message'])
    print('Replication configuration not created')
    raise e

Replication configuration created


* Para recuperar la configuración de replicación, utilice el comando get-bucket-replication.

In [45]:
get_bucket_replication = s3_client.get_bucket_replication(Bucket=bucket_orig)
get_bucket_replication

{'ResponseMetadata': {'RequestId': '93N64SPYAYGZN5Y8',
  'HostId': '9k8wpg8mVw8iWFaKtwDLPeIDQXdAjzTLLSmRhEFwDRChg/6+UBecwGpq3wj8PpiZdCxXKoMpPrk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9k8wpg8mVw8iWFaKtwDLPeIDQXdAjzTLLSmRhEFwDRChg/6+UBecwGpq3wj8PpiZdCxXKoMpPrk=',
   'x-amz-request-id': '93N64SPYAYGZN5Y8',
   'date': 'Sun, 23 Apr 2023 17:24:55 GMT',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ReplicationConfiguration': {'Role': 'arn:aws:iam::305985460937:role/s3-replication-role',
  'Rules': [{'ID': 'YmVhNGY5NGYtZTcwNS00NjQzLWEzOWMtMWQ0OGI0MGRlYmIw',
    'Priority': 1,
    'Filter': {'Prefix': 'Tax'},
    'Status': 'Enabled',
    'Destination': {'Bucket': 'arn:aws:s3:::cookbook-112194'},
    'DeleteMarkerReplication': {'Status': 'Disabled'}}]}}

### Pruebe la configuración en la consola de Amazon S3:

* Inicie sesión en la AWS Management Console y abra la consola de Amazon S3 en https://console.aws.amazon.com/s3/.

* En el bucket *cookbook-orig-XXXXX*, cree una carpeta llamada Tax.

* Agregue objetos de ejemplo a la carpeta Tax en el bucket *cookbook-orig-XXXXX*.

En el bucket *cookbook-dest-XXXXX*, compruebe lo siguiente:

* Que Amazon S3 haya replicado los objetos.

* En las properties (propiedades) del objeto, que Replication Status (Estado de replicación) está establecido en Replica (lo que lo identifica como un objeto de réplica).

* En las propiedades del objeto, que la sección de permisos no muestra ningún permiso. Esto significa que la réplica aún pertenece al propietario del bucket source y que el propietario del bucket *cookbook-dest-XXXXX* no tiene permisos en la réplica del objeto. Puede agregar opciones de configuración adicionales para indicar a Amazon S3 que cambie la titularidad de la réplica. Para ver un ejemplo, consulte [Cambio del propietario de la réplica cuando los buckets de origen y destino son propiedad de diferentes cuentas](https://docs.aws.amazon.com/es_es/AmazonS3/latest/userguide/replication-walkthrough-3.html).